In [1]:
import pickle
import itertools
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [162]:
with open("annotation_one_1_2.curve", "rb") as pickle_file:
    curve_fragment = pickle.load(pickle_file)

In [163]:
cfs = [[],[],[],[],[],[],[],[],[]]
for cols in curve_fragment:
    for i in range(len(cols)):
        if i == 0:
            cfs[0] += cols[i]
        if i == 1:
            cfs[1] += cols[i]
        if i == 2:
            cfs[2] += cols[i]
        if i == 3:
            cfs[3] += cols[i]
        if i == 4:
            cfs[4] += cols[i]
        if i == 5:
            cfs[5] += cols[i]
        if i == 6:
            cfs[6] += cols[i]
        if i == 7:
            cfs[7] += cols[i]
        if i == 8:
            cfs[8] += cols[i]
        if i == 9:
            cfs[8] += cols[i]
        if i == 10:
            cfs[10] += cols[i]
        if i == 11:
            cfs[11] += cols[i]
        if i == 12:
            cfs[12] += cols[i]

IndexError: list index out of range

In [164]:
_cfs = []
for i in cfs:
    if len(i) != 0:
        _cfs += i
cfs = _cfs

In [165]:
len(cfs)

0

In [166]:
_cfs = []
for i in range(len(cfs)):
    if i%10 == 0:
        _cfs += cfs[i].tolist()
cfs = _cfs

In [167]:
pcd = o3d.io.read_point_cloud("polygon/GmJMC025_02_ds_denoise-Copy1.ply")

In [168]:
geoms = [pcd]
#geoms.append(edges_to_lineset(mesh, edges, (1, 0, 0)))

In [169]:
#o3d.visualization.draw_geometries(geoms)

In [170]:
def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry

angle = [np.pi, np.pi, 0] # pcd.rotate()
#angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

In [171]:
np.asarray(pcd.colors)

array([[0.7372549 , 0.74117647, 0.13333333],
       [0.89019608, 0.46666667, 0.76078431],
       [0.85882353, 0.85882353, 0.55294118],
       ...,
       [0.7372549 , 0.74117647, 0.13333333],
       [0.85882353, 0.85882353, 0.55294118],
       [0.85882353, 0.85882353, 0.55294118]])

In [172]:
cfs

[]

In [173]:
curve_fragment = np.array((curve_fragment[0],curve_fragment[2],curve_fragment[1]))

In [174]:
flatten_curve_frag = curve_fragment.T#list(itertools.chain.from_iterable(cfs[1])))
#flatten_curve_frag = (np.linalg.inv(R_mirror)@np.linalg.inv(R)@flatten_curve_frag.T).T
#flatten_curve_frag = np.array(cfs)
point_num = flatten_curve_frag.shape[0]
curve_pcd = o3d.geometry.PointCloud()
curve_pcd.points = o3d.utility.Vector3dVector(flatten_curve_frag)
curve_pcd.colors = o3d.utility.Vector3dVector(np.repeat(np.array((0, 1, 0)).reshape(1,-1), point_num, axis=0))
geoms.append(curve_pcd)

In [175]:
o3d.visualization.draw_geometries(geoms)